## Outdated Information For Future Reference
This file will attempt to perform sentence completion a little differently by utilizing [numpy](https://numpy-ml.readthedocs.io/en/latest/numpy_ml.ngram.goodturing.html) instead of nltk. Other numpy smoothing algorithms can be found [here](https://numpy-ml.readthedocs.io/en/latest/numpy_ml.ngram.html#:~:text=Laplace%20smoothing%20is%20the%20assumption,time%20than%20it%20actually%20does.&text=where%20c(a)%20denotes%20the,n%20%2Dgrams%20in%20the%20corpus.).

## Dataset Processing
The main goal of this notebook is now to investigate alternative processing methods for our input data. This is necessary so we can more easily add more information to the model.

# Input Creation
This file will be used to create and preprocess data used for model training. Our model was originally trained on only transcripts of lectures on Khan Academy. These lectures served us well as a proof of concept but introduced biases and limitations due to being spoken text. We aim to alleviate some of this problem by introducing more information to our model. This will primarily be done by expanding the size of the input corpus used to train the model. Many of the additional dataset sources are too large to include directly into our repository. This is due to the 100mb file size limit and 2GB repository size limit for GitHub. To combat this, while still allowing for more data, we will be using this file to download and setup the files locally. Note that some of the datasets being downloaded as part of this file can reach up to 37GB. This may cause performance issues on slower networks. This file also allows for the addition of ebooks to the model. Ebooks to be used in the model training should be added to the Epubs folder under the Datasets directory.

## General Process Order
1. Download the datasets and unzip if necessary. Additionlly, store each document to its own .txt file if necessary, as is the case with Khan Academy transcripts.
2. Load in the dataset from disk.
3. Run the standardized cleaning method over the text in each file.
4. Store the updated text to disk for use in the primary model training file.

## Datasets
- [Khan Academy Transcripts](https://www.khanacademy.org/)
- [Kaggle Ted Talks Dataset](https://www.kaggle.com/datasets/rounakbanik/ted-talks)
- [1.7GB of Project Gutenberg text files](https://zenodo.org/record/3360392#.ZFU5nXbMIuV)
- [Bookcorpus part 1: 2GB](https://the-eye.eu/public/AI/pile_preliminary_components/)
- [Bookcorpus part 3: 37GB](https://the-eye.eu/public/AI/pile_preliminary_components/)

In [1]:
from pathlib import Path
from collections import Counter, defaultdict
import pandas as pd
import random
from tqdm.notebook import tqdm
import nltk
from nltk.util import ngrams
import nltk.data
import re
import contractions
from bs4 import BeautifulSoup
import unidecode
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
import math
import numpy as np
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import pad_sequence
from nltk.util import everygrams
from nltk.lm import MLE
from functools import partial

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()
encoder = LabelEncoder()
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tqdm.pandas()
n = 3
new_dir = "Datasets/Input"

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Grant\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up

In [17]:
def clean_base(x: str):
        # remove any html tags
        x = BeautifulSoup(x, "html.parser").get_text(separator=" ")
        # # set all to lower
        # x = x.lower()
        # clean up the contractions
        x = contractions.fix(x)
        # remove accended characters
        x = unidecode.unidecode(x)
        # # remove stopwords: https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python
        # x = ' '.join([word for word in x.split() if word not in cachedStopWords]) # slower to use word tokenize
        # # fix punctuation spacing
        # x = re.sub(r'(?<=[\.\,\?])(?=[^\s])', r' ', x)
        # # strip punctuation
        # x = re.sub(r'[\.\,\?\\\/\<\>\;\:\[\]\{\}]', r'', x)
        # strip quotes
        # x = x.replace('\'', '').replace('\"', '')
        x = x.replace('\"', '')
        # # remove some actions
        # remove_list = ['(Laughter)', '(laughter)', '(Music)', '(music)', '(Music ends)', '(Audience cheers)', '(Applause)', '(Applause ends)', '(Applause continues)', '(Bells)', '(Trumpet)', '(Clears throat)']
        # x = ' '.join([word for word in x.split() if word not in remove_list])
        # remove extraneous items
        x = x.replace(' -- ', '').replace(' .. ', ' ').replace(' ... ', ' ')
        # remove extra whitespace
        x = ' '.join(x.strip().split())
        # # may want to add lematization
        # x = ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
        # remove some of the extra bracket tags
        x = re.sub(r"\s{2,}", " ", re.sub(r"[\(\[\{][^\)\]\}]*[\)\]\}]", "", x))
        # # Strip newlines
        x = re.sub(r"\n", " ", x)
        return x

## Convert Khan
The first step is to change how we are currently storing the khan academy data. Instead of storing the transcripts inside of csv files, or as one newline delmited text file, we instead will be creating a new text file for each transcript. These files will all be stored in the same directory so we can load them all with glob. The names of each file will be based on the video title and the course of the lecture.

In [3]:
computing_df = pd.read_csv(Path("Datasets/KhanAcademy/Computing.csv"))
computing_df = computing_df.dropna()

economics_df = pd.read_csv(Path("Datasets/KhanAcademy/Economics.csv"))
economics_df = economics_df.dropna()

humanities_df = pd.read_csv(Path("Datasets/KhanAcademy/Humanities.csv"))
humanities_df = humanities_df.dropna()

math_df = pd.read_csv(Path("Datasets/KhanAcademy/Math.csv"))
math_df = math_df.dropna()

science_df = pd.read_csv(Path("Datasets/KhanAcademy/Science.csv"))
science_df = science_df.dropna()

khan_dfs = [computing_df, economics_df, humanities_df, math_df, science_df]
khan = pd.concat(khan_dfs, axis=0)
khan.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8261 entries, 0 to 2789
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       8261 non-null   object
 1   unit         8261 non-null   object
 2   lesson       8261 non-null   object
 3   video_title  8261 non-null   object
 4   about        8261 non-null   object
 5   transcript   8261 non-null   object
dtypes: object(6)
memory usage: 451.8+ KB


In [4]:
def store_khan_lecture(row):
    new_title = f"khan - {row['course']} - {row['video_title']}"
    new_title = re.sub(r'[\.\,\?\\\/\<\>\;\:\[\]\{\}\!\"®︎\|\*\(\)]', r'', new_title)
    new_fp = Path(f"{new_dir}/{new_title}.txt")
    with open(new_fp, 'w', encoding="utf-8") as f:
        f.write(row['transcript'])
        # f.write(clean_base(lecture['transcript']))

khan.progress_apply(lambda row: store_khan_lecture(row), axis=1)
# for lecture in khan.iloc:
#     store_khan_lecture(lecture)

  0%|          | 0/8261 [00:00<?, ?it/s]

0       None
1       None
5       None
6       None
7       None
        ... 
2785    None
2786    None
2787    None
2788    None
2789    None
Length: 8261, dtype: object

## Determine how to read data back in
This section investigates how we can read this data back into the program for use.

In [5]:
directory_path = Path(new_dir)
documents = []
for data_path in tqdm(list(directory_path.glob("**/*.txt"))):
    text = ""
    with open(Path(data_path), "r", encoding="utf-8") as f:
        text = clean_base(f.read())
        documents.append(text)

  0%|          | 0/8067 [00:00<?, ?it/s]

Additional text files were sourced from [Project Gutenberg](https://www.gutenberg.org/) indirectly through [Zenodo](https://zenodo.org/record/3360392#.ZFUirnbMIuU).

## Ebooks:
the below section investigates how to read ebooks with an OCHEM ebook I found online. I investigated two options for this. The first option is more concise, but has issues where it removes the spaces between some words during html parsing. The second option is a little slower but more accurate so we will go with that.

In [36]:
import ebooklib
from ebooklib import epub

book = epub.read_epub(Path("Datasets/Epubs/Organic-Chemistry-I-1639153167.epub"))

text = ""
for doc in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
    # print(doc.content)
    text += clean_base(doc.content)
# print(text)

c:\Python310\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [24]:
# !pip install epub-conversion ebooklib xml_cleaner

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for xml_cleaner: started
  Running setup.py install for xml_cleaner: finished with status 'done'


  DEPRECATION: xml_cleaner is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
# from epub_conversion import Converter
# converter = Converter(Path("Datasets/Epubs/"))
# converter.convert("epubs.txt")
from epub_conversion.utils import open_book, convert_epub_to_lines

directory_path = Path("Datasets/Epubs")
documents = []
for data_path in tqdm(list(directory_path.glob("**/*.epub"))):
    book = open_book(Path(data_path))
    lines = [clean_base(line) for line in convert_epub_to_lines(book)]
    lines = [i for i in lines if i != '']
    text = '\n'.join(lines)
    with open(Path(f'{new_dir}/{data_path.stem}.txt'), 'w', encoding='utf-8') as f:
        f.write(text)

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Python310\lib\site-packages\epub\__init__.py:139: SyntaxWarning: The ePub does not define any NCX file
  warnings.warn('The ePub does not define any NCX file',
